In [23]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

app_packages = [
    'com.google.android.apps.meetings',
    'com.instagram.android',
    'com.microsoft.teams',
    'com.netflix.mediaclient',
    'com.zhiliaoapp.musically',
    'com.whatsapp',
    'us.zoom.videomeetings'
]

app_infos = []

for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='us')
  del info['comments']
  app_infos.append(info)    

app_infos_df = pd.DataFrame(app_infos)
app_infos_df.to_csv('apps.csv', index=None, header=True)

app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 10000,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('reviews.csv', index=None, header=True)

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [42:37<00:00, 365.32s/it]


In [2]:
import pandas as pd
data = pd.read_csv("reviews_instagram_v2.csv")
data_text = data[['content']]
data_text['index'] = data_text.index
documents = data_text

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
import numpy as np

def lemmatize_only(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(str(text).encode('ascii',errors='ignore').decode()):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

processed_docs = documents['content'].map(preprocess)
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=5000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

print('\nBag of Words Model:\n')
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

print('\nTF-IDF Model:\n')
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))


Bag of Words Model:

Topic: 0 
Words: 0.070*"amazing" + 0.021*"story" + 0.019*"video" + 0.018*"stories" + 0.017*"upload" + 0.017*"time" + 0.015*"update" + 0.012*"work" + 0.012*"phone" + 0.011*"instagram"
Topic: 1 
Words: 0.173*"best" + 0.043*"excellent" + 0.042*"account" + 0.031*"instagram" + 0.022*"video" + 0.020*"time" + 0.018*"perfect" + 0.014*"download" + 0.014*"help" + 0.012*"world"
Topic: 2 
Words: 0.569*"good" + 0.044*"cool" + 0.017*"experience" + 0.012*"insta" + 0.011*"beautiful" + 0.010*"filters" + 0.010*"like" + 0.009*"application" + 0.009*"enjoy" + 0.009*"videos"
Topic: 3 
Words: 0.063*"post" + 0.044*"people" + 0.028*"superb" + 0.026*"follow" + 0.016*"want" + 0.016*"useful" + 0.015*"page" + 0.015*"like" + 0.015*"dont" + 0.014*"know"
Topic: 4 
Words: 0.172*"love" + 0.103*"great" + 0.061*"super" + 0.048*"instagram" + 0.021*"friends" + 0.016*"wonderful" + 0.012*"easy" + 0.011*"share" + 0.009*"people" + 0.009*"star"
Topic: 5 
Words: 0.121*"like" + 0.051*"posts" + 0.018*"follow"